### カリキュラム - 【AIコース】CNNによる画像分類 実装5

#### skip connection
##### ・前処理

In [1]:
import keras
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
img_rows = X_train[0].shape[1]
img_cols = X_train[0].shape[0]
num_classes = len(set(y_train))

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = X_train.shape[1:4]

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.
/Users/yasu/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


##### ・モデル構築

In [10]:
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, merge
from keras.models import Model

img_input = Input(shape = input_shape)
x = Conv2D(1,
           kernel_size = (3, 3),
           strides = (1, 1),
           padding = "same",
           activation = "relu",
           input_shape = input_shape)(img_input)
x = MaxPooling2D(pool_size = (2, 2),
                 strides = (2, 2),
                 padding = "same")(x)
x = Conv2D(1,
           kernel_size = (3, 3),
           strides = (1, 1),
           padding = "same",
           activation = "relu")(x)
x = Conv2D(1,
           kernel_size = (3, 3),
           strides = (1, 1),
           padding = "same",
           activation = "relu")(x)
# z = merge([x, y], mode = "sum")
x = MaxPooling2D(pool_size = (2, 2),
                 strides = (2, 2),
                 padding = "same")(x)
x = Flatten()(x)
x = Dense(128, activation = "relu")(x)
x = Dense(num_classes, activation = "softmax")(x)

model = Model(img_input, x)

model.compile(loss = keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.Adadelta(),
              metrics = ["accuracy"])

model.fit(X_train, y_train,
          batch_size = 128,
          epochs = 3,
          verbose = 1,
          validation_data = (X_test, y_test))
score = model.evaluate(X_test, y_test, verbose = 1)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 12s 199us/step - loss: 0.6547 - acc: 0.7901 - val_loss: 0.2741 - val_acc: 0.9183